In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# ---------------------------------------------
# STEP 0: SETUP
# ---------------------------------------------
import pandas as pd
import numpy as np
import nltk
import string
import openai
from nltk import word_tokenize, pos_tag
from textblob import TextBlob
import matplotlib.pyplot as plt

nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('stopwords')
from nltk.corpus import stopwords

# Set OpenAI API key
with open("/content/drive/MyDrive/Colab/DAI_AID/keys/openai_key.txt") as f:
    openai.api_key = f.read().strip()

# ---------------------------------------------
# STEP 1: CONFIG
# ---------------------------------------------
base_dir = "/content/drive/MyDrive/Colab/DAI_AID/extracted_data/cleaned_data"
files = [
    "airpodsmax_youtube_cleaned.csv",
    "sony_xm5_youtube_cleaned.csv",
    "sennheiser_momentum4_youtube_cleaned.csv",
    "airpodsmax_reddit_cleaned.csv",
    "sony_xm5_reddit_cleaned.csv",
    "sennheiser_momentum4_reddit_cleaned.csv",
]

filename_to_product = {
    "airpodsmax_youtube_cleaned.csv": "AirPods Max",
    "sony_xm5_youtube_cleaned.csv": "Sony XM5",
    "sennheiser_momentum4_youtube_cleaned.csv": "Momentum 4",
    "airpodsmax_reddit_cleaned.csv": "AirPods Max",
    "sony_xm5_reddit_cleaned.csv": "Sony XM5",
    "sennheiser_momentum4_reddit_cleaned.csv": "Momentum 4",
}

valid_specs = {
    "battery", "noise", "cancellation", "comfort", "sound", "fit", "build", "design",
    "durability", "case", "controls", "bluetooth", "microphone", "charging", "portability",
    "volume", "latency", "app", "touch", "connectivity", "price", "weight"
}

noise = set(stopwords.words("english")).union(
    {"airpods", "apple", "sony", "sennheiser", "bruh", "max", "xm5", "momentum", "lol", "bro"}
)

def get_sentiment(text):
    if not isinstance(text, str): return "Neutral"
    polarity = TextBlob(text).sentiment.polarity
    return "Positive" if polarity > 0.1 else "Negative" if polarity < -0.1 else "Neutral"

def extract_keywords(text):
    if not isinstance(text, str):
        return []
    tokens = word_tokenize(text.lower())
    tagged = pos_tag(tokens)
    return [w for w, t in tagged if t.startswith("NN") and w not in noise and w not in string.punctuation]


# ---------------------------------------------
# STEP 2: LOAD + PROCESS FILES
# ---------------------------------------------
all_rows = []
for file in files:
    path = f"{base_dir}/{file}"
    df = pd.read_csv(path)
    if 'cleaned_comment' not in df.columns:
        print(f"Skipping {file}")
        continue
    for _, row in df.iterrows():
        sent = get_sentiment(row['cleaned_comment'])
        kws = extract_keywords(row['cleaned_comment'])
        all_rows.append({
            "filename": file,
            "product": filename_to_product[file],
            "comment": row['cleaned_comment'],
            "sentiment": sent,
            "keywords": kws
        })

df_results = pd.DataFrame(all_rows)
df_results["sentiment_score"] = df_results["sentiment"].map({"Positive": 1, "Neutral": 0, "Negative": -1})

# ---------------------------------------------
# STEP 3: GAP ANALYSIS
# ---------------------------------------------
df_exploded = df_results.explode("keywords")
df_exploded["keywords"] = df_exploded["keywords"].str.strip().str.lower()
df_specs = df_exploded[df_exploded["keywords"].isin(valid_specs)].copy()

pivot_df = df_specs.groupby(["keywords", "product"])["sentiment_score"].mean().reset_index()
pivot_table = pivot_df.pivot(index="keywords", columns="product", values="sentiment_score").fillna(0)

pivot_table["CompetitorAvg"] = pivot_table[["Sony XM5", "Momentum 4"]].mean(axis=1)
pivot_table["Gap_vs_Competitors"] = pivot_table["CompetitorAvg"] - pivot_table["AirPods Max"]
pivot_table["Underperforming"] = pivot_table["Gap_vs_Competitors"] > 0.2

price_map = {"AirPods Max": 549, "Sony XM5": 399, "Momentum 4": 379}
for product in price_map:
    pivot_table[f"{product}_P2P"] = pivot_table[product] / price_map[product]

# Display gap table
gap_cols = [
    "AirPods Max", "Sony XM5", "Momentum 4",
    "CompetitorAvg", "Gap_vs_Competitors",
    "Underperforming",
    "AirPods Max_P2P", "Sony XM5_P2P", "Momentum 4_P2P"
]
print("\n=== GAP ANALYSIS TABLE ===")
display(pivot_table[gap_cols].sort_values("Gap_vs_Competitors", ascending=False))

# ---------------------------------------------
# STEP 4: DESIGN OPPORTUNITY EXPLAINER (GPT)
# ---------------------------------------------
underperf = pivot_table[pivot_table["Underperforming"]].index.tolist()
comments_by_feature = df_specs[
    (df_specs["product"] == "AirPods Max") &
    (df_specs["keywords"].isin(underperf))
].groupby("keywords")["comment"].apply(lambda x: "\n".join(x.dropna().unique()[:10]))

def gpt_design_opportunity(feature, comments):
    prompt = f"""
You are a product strategist. Users gave negative feedback on the **"{feature}"** of the AirPods Max.
Here are sample complaints:
{comments}

Based on these:
1. Identify specific issues
2. Translate to measurable design specs
3. Suggest improvements
4. Estimate potential impact (sales, brand, ROI)
5. List all insights clearly in bullet points.
"""
    try:
        response = openai.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.7,
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {e}"

# Generate output
for feature, sample_comments in comments_by_feature.items():
    print(f"\n\n===== DESIGN OPPORTUNITY: {feature.upper()} =====")
    print(gpt_design_opportunity(feature, sample_comments))


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.



=== GAP ANALYSIS TABLE ===


product,AirPods Max,Sony XM5,Momentum 4,CompetitorAvg,Gap_vs_Competitors,Underperforming,AirPods Max_P2P,Sony XM5_P2P,Momentum 4_P2P
keywords,,,,,,,,,
microphone,0.333333,0.473684,0.875000,0.674342,0.341009,True,0.000607,0.001187,0.002309
fit,0.571429,0.851852,0.846154,0.849003,0.277574,True,0.001041,0.002135,0.002233
build,0.529412,0.800000,0.777778,0.788889,0.259477,True,0.000964,0.002005,0.002052
latency,0.161290,0.318182,0.500000,0.409091,0.247801,True,0.000294,0.000797,0.001319
comfort,0.555556,0.742857,0.782609,0.762733,0.207177,True,0.001012,0.001862,0.002065
case,0.403587,0.471649,0.620253,0.545951,0.142364,False,0.000735,0.001182,0.001637
battery,0.475410,0.703448,0.523077,0.613263,0.137853,False,0.000866,0.001763,0.001380
weight,0.378378,0.666667,0.363636,0.515152,0.136773,False,0.000689,0.001671,0.000959
price,0.481928,0.579545,0.592105,0.585825,0.103898,False,0.000878,0.001452,0.001562




===== DESIGN OPPORTUNITY: BUILD =====
Specific issues:
- Comfort issues such as ear heating up and uncomfortable pressure after extended use
- Durability concerns including condensation buildup and hinge noise
- Portability issues with the Smart Case not meeting expectations
- Build quality comparisons with other brands such as Beats and concerns about plastic build
- Fit issues with the headphones sliding around on the head with movement

Measurable design specs:
- Improve earpad material to prevent overheating and discomfort
- Address condensation buildup issue and ensure hinge noise is eliminated
- Redesign Smart Case for better functionality and protection
- Enhance build quality with stronger materials and better construction
- Improve headband design for better fit and stability during movement

Improvements:
- Use more breathable materials for earpads to prevent overheating
- Implement better sealant to prevent condensation buildup
- Reinforce hinge mechanism to reduce noise a

In [ ]:
THIS IS WHAT WE ARE USING
# ---------------------------------------------
# STEP 0: SETUP
# ---------------------------------------------
import pandas as pd
import numpy as np
import nltk
import string
import openai
from nltk import word_tokenize, pos_tag
from textblob import TextBlob
import matplotlib.pyplot as plt

nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('stopwords')
from nltk.corpus import stopwords

# Set OpenAI API key
with open("/content/drive/MyDrive/Colab/DAI_AID/keys/openai_key.txt") as f:
    openai.api_key = f.read().strip()

# ---------------------------------------------
# STEP 1: CONFIG
# ---------------------------------------------
base_dir = "/content/drive/MyDrive/Colab/DAI_AID/extracted_data/cleaned_data"
files = [
    "airpodsmax_youtube_cleaned.csv",
    "sony_xm5_youtube_cleaned.csv",
    "sennheiser_momentum4_youtube_cleaned.csv",
    "airpodsmax_reddit_cleaned.csv",
    "sony_xm5_reddit_cleaned.csv",
    "sennheiser_momentum4_reddit_cleaned.csv",
]

filename_to_product = {
    "airpodsmax_youtube_cleaned.csv": "AirPods Max",
    "sony_xm5_youtube_cleaned.csv": "Sony XM5",
    "sennheiser_momentum4_youtube_cleaned.csv": "Momentum 4",
    "airpodsmax_reddit_cleaned.csv": "AirPods Max",
    "sony_xm5_reddit_cleaned.csv": "Sony XM5",
    "sennheiser_momentum4_reddit_cleaned.csv": "Momentum 4",
}

valid_specs = {
    "battery", "noise", "cancellation", "comfort", "sound", "fit", "build", "design",
    "durability", "case", "controls", "bluetooth", "microphone", "charging", "portability",
    "volume", "latency", "app", "touch", "connectivity", "price", "weight"
}

noise = set(stopwords.words("english")).union(
    {"airpods", "apple", "sony", "sennheiser", "bruh", "max", "xm5", "momentum", "lol", "bro"}
)

def get_sentiment(text):
    if not isinstance(text, str): return "Neutral"
    polarity = TextBlob(text).sentiment.polarity
    return "Positive" if polarity > 0.1 else "Negative" if polarity < -0.1 else "Neutral"

def extract_keywords(text):
    if not isinstance(text, str):
        return []
    tokens = word_tokenize(text.lower())
    tagged = pos_tag(tokens)
    return [w for w, t in tagged if t.startswith("NN") and w not in noise and w not in string.punctuation]


# ---------------------------------------------
# STEP 2: LOAD + PROCESS FILES
# ---------------------------------------------
all_rows = []
for file in files:
    path = f"{base_dir}/{file}"
    df = pd.read_csv(path)
    if 'cleaned_comment' not in df.columns:
        print(f"Skipping {file}")
        continue
    for _, row in df.iterrows():
        sent = get_sentiment(row['cleaned_comment'])
        kws = extract_keywords(row['cleaned_comment'])
        all_rows.append({
            "filename": file,
            "product": filename_to_product[file],
            "comment": row['cleaned_comment'],
            "sentiment": sent,
            "keywords": kws
        })

df_results = pd.DataFrame(all_rows)
df_results["sentiment_score"] = df_results["sentiment"].map({"Positive": 1, "Neutral": 0, "Negative": -1})

# ---------------------------------------------
# STEP 3: GAP ANALYSIS
# ---------------------------------------------
df_exploded = df_results.explode("keywords")
df_exploded["keywords"] = df_exploded["keywords"].str.strip().str.lower()
df_specs = df_exploded[df_exploded["keywords"].isin(valid_specs)].copy()

pivot_df = df_specs.groupby(["keywords", "product"])["sentiment_score"].mean().reset_index()
pivot_table = pivot_df.pivot(index="keywords", columns="product", values="sentiment_score").fillna(0)

pivot_table["CompetitorAvg"] = pivot_table[["Sony XM5", "Momentum 4"]].mean(axis=1)
pivot_table["Gap_vs_Competitors"] = pivot_table["CompetitorAvg"] - pivot_table["AirPods Max"]
pivot_table["Underperforming"] = pivot_table["Gap_vs_Competitors"] > 0.2

price_map = {"AirPods Max": 549, "Sony XM5": 399, "Momentum 4": 379}
for product in price_map:
    pivot_table[f"{product}_P2P"] = pivot_table[product] / price_map[product]

# Display gap table
gap_cols = [
    "AirPods Max", "Sony XM5", "Momentum 4",
    "CompetitorAvg", "Gap_vs_Competitors",
    "Underperforming",
    "AirPods Max_P2P", "Sony XM5_P2P", "Momentum 4_P2P"
]
print("\n=== GAP ANALYSIS TABLE ===")
display(pivot_table[gap_cols].sort_values("Gap_vs_Competitors", ascending=False))

# ---------------------------------------------
# STEP 4: DESIGN OPPORTUNITY INSIGHTS (TOP GAP ONLY)
# ---------------------------------------------
# Sort underperforming specs by highest gap
N = 5  # adjust how many top underperforming specs to analyze
top_underperf = (
    pivot_table[pivot_table["Underperforming"]]
    .sort_values("Gap_vs_Competitors", ascending=False)
    .head(N)
    .index
    .tolist()
)

# Get AirPods Max comments for those specs
comments_by_feature = df_specs[
    (df_specs["product"] == "AirPods Max") &
    (df_specs["keywords"].isin(top_underperf))
].groupby("keywords")["comment"].apply(lambda x: "\n".join(x.dropna().unique()[:10]))

def gpt_design_opportunity(feature, comments):
    prompt = f"""
You are a product strategist. Users gave negative feedback on the **"{feature}"** of the AirPods Max.
Here are sample complaints:
{comments}

Based on these:
1. Identify specific issues
2. Translate to measurable design specs
3. Suggest improvements
4. Estimate potential impact (sales, brand, ROI)
5. List all insights clearly in bullet points.
"""
    try:
        response = openai.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.7,
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {e}"

# Generate insights for top N underperforming specs
print(f"\n=== GPT DESIGN OPPORTUNITY ANALYSIS (TOP {N} FEATURES) ===")
for feature, sample_comments in comments_by_feature.items():
    print(f"\n\n===== DESIGN OPPORTUNITY: {feature.upper()} =====")
    print(gpt_design_opportunity(feature, sample_comments))


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!



=== GAP ANALYSIS TABLE ===


product,AirPods Max,Sony XM5,Momentum 4,CompetitorAvg,Gap_vs_Competitors,Underperforming,AirPods Max_P2P,Sony XM5_P2P,Momentum 4_P2P
keywords,,,,,,,,,
microphone,0.333333,0.473684,0.875000,0.674342,0.341009,True,0.000607,0.001187,0.002309
fit,0.571429,0.851852,0.846154,0.849003,0.277574,True,0.001041,0.002135,0.002233
build,0.529412,0.800000,0.777778,0.788889,0.259477,True,0.000964,0.002005,0.002052
latency,0.161290,0.318182,0.500000,0.409091,0.247801,True,0.000294,0.000797,0.001319
comfort,0.555556,0.742857,0.782609,0.762733,0.207177,True,0.001012,0.001862,0.002065
case,0.403587,0.471649,0.620253,0.545951,0.142364,False,0.000735,0.001182,0.001637
battery,0.475410,0.703448,0.523077,0.613263,0.137853,False,0.000866,0.001763,0.001380
weight,0.378378,0.666667,0.363636,0.515152,0.136773,False,0.000689,0.001671,0.000959
price,0.481928,0.579545,0.592105,0.585825,0.103898,False,0.000878,0.001452,0.001562



=== GPT DESIGN OPPORTUNITY ANALYSIS (TOP 5 FEATURES) ===


===== DESIGN OPPORTUNITY: BUILD =====
1. Specific issues:
- Complaints about the build quality of the AirPods Max, including issues with the case, ear pad comfort, durability, portability, and fit on the head.
- Comparison to Beats headphones in terms of build quality, sound signature, and affordability.
- Concerns about the durability and reliability of Beats headphones.
- Desire for lightweight design with sturdy build quality.
- Preference for replaceable ear pads and design elements from other headphone brands like Sennheiser and Master & Dynamic.

2. Measurable design specs:
- Ear pad comfort rating of 7.5/10, with a goal to improve comfort for extended wear.
- Durability and build rating of 9.0/10, with a focus on addressing condensation buildup, hinge noise, and overall sturdiness.
- Portability rating of 6/10, with a need to improve the functionality and user experience of the Smart Case.
- Sound quality rating of 8.5/


---

## 🔍 Full Breakdown of the Code

---

### **STEP 0: SETUP**
- **Imports** required libraries like `pandas`, `openai`, `nltk`, `textblob`, etc.
- Downloads NLTK resources needed for tokenization and POS tagging.
- Reads your OpenAI API key securely from Drive.

---

### **STEP 1: CONFIG**
- Defines the **list of cleaned CSV files** from Reddit and YouTube.
- Maps each file to its **corresponding product** (`AirPods Max`, `Sony XM5`, or `Momentum 4`).
- Defines a **set of valid specs/features** to focus on (e.g., battery, weight, price).
- Builds a **noise filter** that removes common stopwords and irrelevant slang or brand words from the keyword extraction step.

---

### **STEP 2: LOAD + PROCESS FILES**
- Iterates through each cleaned CSV file.
- For each comment:
  - Performs **sentiment analysis** using TextBlob (`Positive`, `Neutral`, `Negative`).
  - Performs **keyword extraction**, keeping only nouns that match valid specs and are not in the noise list.
- Results are stored in a single unified dataframe `df_results`.

---

### **STEP 3: GAP ANALYSIS**
- Explodes keywords so each row has one keyword per comment.
- Filters to retain only keywords that are valid specs.
- **Groups sentiment scores** by `product` and `feature`.
- Creates a **pivot table** showing average sentiment score per spec per product.
- Computes:
  - **Competitor Average** = average of Sony XM5 and Momentum 4 scores
  - **Gap_vs_Competitors** = how much worse AirPods Max performs relative to the competition
  - **Underperforming = True** if gap exceeds 0.2
- Adds **price-to-performance (P2P)** scores by dividing sentiment by product price.

---

### **STEP 4: GPT DESIGN OPPORTUNITIES**
- Picks **top N underperforming specs** based on the highest `Gap_vs_Competitors`.
- For each one, collects up to 10 real comments mentioning that spec.
- Feeds those into GPT-3.5-turbo via OpenAI API with a detailed prompt asking for:
  - User problems
  - Measurable design issues
  - Suggestions
  - ROI implications
- GPT response is printed as a **feature-specific design insight block**.

---

## 📊 Explanation for Use in Project Report

---

### **Objective:**
To identify actionable design opportunities for the AirPods Max by analyzing real user feedback from Reddit and YouTube and benchmarking it against top competitors (Sony XM5, Sennheiser Momentum 4).

---

### **Why This Analysis Matters:**

1. **Grounds design decisions in user sentiment:**
   - Instead of relying on intuition or vague reviews, we quantify how users feel about specific product features (e.g., weight, comfort, battery life).
   - This allows product teams to prioritize real pain points.

2. **Benchmarks against market leaders:**
   - By comparing AirPods Max to Sony and Sennheiser, we identify where the product **underperforms in the eyes of users**, even though it has a higher price point.

3. **Pinpoints features with measurable gaps:**
   - The “Gap_vs_Competitors” metric directly highlights features where sentiment is significantly more negative for AirPods Max than its competitors.

4. **Justifies ROI-driven redesigns:**
   - Using GPT, we translate complaints into **design requirements** (e.g., “uncomfortable” → adjust headband pressure).
   - Each suggestion is accompanied by potential **business impact** (e.g., retention, revenue, or brand perception).

5. **Makes funding decisions easier:**
   - If a design flaw (e.g., “heavy build”) appears across hundreds of comments and GPT suggests a fix, the PM can argue:
     > “Spending $2k on redesigning this component could lead to a significant boost in user satisfaction and market competitiveness.”

---

### **How This Helps a Product Manager:**
| Value Added | Description |
|-------------|-------------|
| **Prioritization** | Focus on specs that matter most to users and where the product underdelivers. |
| **Cost-benefit clarity** | Identify fixes that bring high sentiment improvement with minimal cost (via P2P and GPT analysis). |
| **Evidence-based justification** | Provides hard data and real quotes to justify design and engineering changes. |
| **Strategic alignment** | Aligns product roadmap with competitive intelligence and user expectations. |

---

### 🔚 Final Note
This analysis transforms scattered social media feedback into **actionable intelligence** for product, design, and business teams — making it easier to decide **where to invest, what to fix, and how to win back market edge**.

---




In [ ]:
# ---------------------------------------------
# SETUP
# ---------------------------------------------
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import matplotlib.pyplot as plt
import re
from textblob import TextBlob

# ---------------------------------------------
# LOAD CLEANED DATA
# ---------------------------------------------
base = "/content/drive/MyDrive/Colab/DAI_AID/extracted_data/cleaned_data"

df_airpods = pd.concat([
    pd.read_csv(f"{base}/airpodsmax_youtube_cleaned.csv"),
    pd.read_csv(f"{base}/airpodsmax_reddit_cleaned.csv")
])
df_sony = pd.concat([
    pd.read_csv(f"{base}/sony_xm5_youtube_cleaned.csv"),
    pd.read_csv(f"{base}/sony_xm5_reddit_cleaned.csv")
])
df_senn = pd.concat([
    pd.read_csv(f"{base}/sennheiser_momentum4_youtube_cleaned.csv"),
    pd.read_csv(f"{base}/sennheiser_momentum4_reddit_cleaned.csv")
])

# ---------------------------------------------
# DEFINE FEATURES
# ---------------------------------------------
feature_keywords = {
    "noise_cancellation": ["noise cancellation", "anc", "active noise"],
    "sound_quality": ["sound quality", "audio", "clarity", "bass", "treble"],
    "comfort": ["comfort", "comfortable", "wearing", "fit", "ear cups"],
    "battery": ["battery", "charge", "charging", "battery life"],
    "build": ["build", "material", "durable", "design", "headband"],
    "price": ["price", "expensive", "worth", "overpriced", "value"]
}

# ---------------------------------------------
# SENTIMENT ANALYSIS BY FEATURE
# ---------------------------------------------
def compute_feature_sentiment(df, name):
    df = df.copy()
    df['sentiment'] = df['cleaned_comment'].apply(lambda text: TextBlob(str(text)).sentiment.polarity)
    for feature in feature_keywords:
        df[feature] = df['cleaned_comment'].apply(
            lambda text: any(re.search(rf"\b{k}\b", str(text).lower()) for k in feature_keywords[feature])
        )
    result = {feature: df[df[feature]]['sentiment'].mean() for feature in feature_keywords}
    return pd.DataFrame.from_dict(result, orient='index', columns=[name])

sent_airpods = compute_feature_sentiment(df_airpods, "AirPods Max")
sent_sony = compute_feature_sentiment(df_sony, "Sony XM5")
sent_senn = compute_feature_sentiment(df_senn, "Momentum 4")

sentiment_combined = pd.concat([sent_airpods, sent_sony, sent_senn], axis=1).sort_index()

# ---------------------------------------------
# FREQUENCY ANALYSIS BY FEATURE (optional)
# ---------------------------------------------
def compute_feature_frequency(df, name):
    df = df.copy()
    counts = {}
    for feature in feature_keywords:
        counts[feature] = df['cleaned_comment'].apply(
            lambda text: any(re.search(rf"\b{k}\b", str(text).lower()) for k in feature_keywords[feature])
        ).sum()
    return pd.DataFrame.from_dict({name: counts})

# ---------------------------------------------
# GAP ANALYSIS
# ---------------------------------------------
gap_df = sentiment_combined.copy()
gap_df["CompetitorAvg"] = gap_df[["Sony XM5", "Momentum 4"]].mean(axis=1)
gap_df["Gap_vs_Competitors"] = gap_df["CompetitorAvg"] - gap_df["AirPods Max"]
gap_df["Underperforming"] = gap_df["Gap_vs_Competitors"] > 0.2

# Price-to-performance (P2P)
price_map = {"AirPods Max": 549, "Sony XM5": 399, "Momentum 4": 379}
for brand in price_map:
    gap_df[f"{brand}_P2P"] = gap_df[brand] / price_map[brand]

# ---------------------------------------------
# FINAL GAP ANALYSIS TABLE
# ---------------------------------------------
gap_output = gap_df.sort_values("Gap_vs_Competitors", ascending=False)

print("\n=== GAP ANALYSIS TABLE ===")
display(gap_output)

# ---------------------------------------------
# LIST UNDERPERFORMING FEATURES
# ---------------------------------------------
underperforming_features = gap_output[gap_output["Underperforming"]].index.tolist()
print("\n=== UNDERPERFORMING FEATURES (AirPods Max vs Competitors) ===")
for feature in underperforming_features:
    print("-", feature)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

=== GAP ANALYSIS TABLE ===


,AirPods Max,Sony XM5,Momentum 4,CompetitorAvg,Gap_vs_Competitors,Underperforming,AirPods Max_P2P,Sony XM5_P2P,Momentum 4_P2P
price,0.119971,0.182845,0.175495,0.179170,0.059199,False,0.000219,0.000458,0.000463
battery,0.141069,0.199425,0.161048,0.180237,0.039167,False,0.000257,0.000500,0.000425
sound_quality,0.172110,0.187758,0.226081,0.206919,0.034809,False,0.000313,0.000471,0.000597
build,0.116800,0.130130,0.117206,0.123668,0.006867,False,0.000213,0.000326,0.000309
comfort,0.223477,0.206079,0.239511,0.222795,-0.000682,False,0.000407,0.000516,0.000632
noise_cancellation,0.256006,0.217378,0.218581,0.217979,-0.038026,False,0.000466,0.000545,0.000577



=== UNDERPERFORMING FEATURES (AirPods Max vs Competitors) ===


In [ ]:
# ---------------------------------------------
# STEP 0: SETUP
# ---------------------------------------------
import pandas as pd
import numpy as np
import nltk
import string
import openai
from nltk import word_tokenize, pos_tag
from textblob import TextBlob
import matplotlib.pyplot as plt

nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('stopwords')
from nltk.corpus import stopwords

# Set OpenAI API key
with open("/content/drive/MyDrive/Colab/DAI_AID/keys/openai_key.txt") as f:
    openai.api_key = f.read().strip()

# ---------------------------------------------
# STEP 1: CONFIG
# ---------------------------------------------
base_dir = "/content/drive/MyDrive/Colab/DAI_AID/extracted_data/cleaned_data"
files = [
    "airpodsmax_youtube_cleaned.csv",
    "sony_xm5_youtube_cleaned.csv",
    "sennheiser_momentum4_youtube_cleaned.csv",
    "airpodsmax_reddit_cleaned.csv",
    "sony_xm5_reddit_cleaned.csv",
    "sennheiser_momentum4_reddit_cleaned.csv",
]

filename_to_product = {
    "airpodsmax_youtube_cleaned.csv": "AirPods Max",
    "sony_xm5_youtube_cleaned.csv": "Sony XM5",
    "sennheiser_momentum4_youtube_cleaned.csv": "Momentum 4",
    "airpodsmax_reddit_cleaned.csv": "AirPods Max",
    "sony_xm5_reddit_cleaned.csv": "Sony XM5",
    "sennheiser_momentum4_reddit_cleaned.csv": "Momentum 4",
}

valid_specs = {
    "battery", "noise", "cancellation", "comfort", "sound", "fit", "build", "design",
    "durability", "case", "controls", "bluetooth", "microphone", "charging", "portability",
    "volume", "latency", "app", "touch", "connectivity", "price", "weight"
}

noise = set(stopwords.words("english")).union(
    {"airpods", "apple", "sony", "sennheiser", "bruh", "max", "xm5", "momentum", "lol", "bro"}
)

def get_sentiment(text):
    if not isinstance(text, str): return "Neutral"
    polarity = TextBlob(text).sentiment.polarity
    return "Positive" if polarity > 0.1 else "Negative" if polarity < -0.1 else "Neutral"

def extract_keywords(text):
    if not isinstance(text, str):
        return []
    tokens = word_tokenize(text.lower())
    tagged = pos_tag(tokens)
    return [w for w, t in tagged if t.startswith("NN") and w not in noise and w not in string.punctuation]

# ---------------------------------------------
# STEP 2: LOAD + PROCESS FILES
# ---------------------------------------------
all_rows = []
for file in files:
    path = f"{base_dir}/{file}"
    df = pd.read_csv(path)
    if 'cleaned_comment' not in df.columns:
        print(f"Skipping {file}")
        continue
    for _, row in df.iterrows():
        sent = get_sentiment(row['cleaned_comment'])
        kws = extract_keywords(row['cleaned_comment'])
        all_rows.append({
            "filename": file,
            "product": filename_to_product[file],
            "comment": row['cleaned_comment'],
            "sentiment": sent,
            "keywords": kws
        })

df_results = pd.DataFrame(all_rows)
df_results["sentiment_score"] = df_results["sentiment"].map({"Positive": 1, "Neutral": 0, "Negative": -1})

# ---------------------------------------------
# STEP 3: GAP ANALYSIS
# ---------------------------------------------
df_exploded = df_results.explode("keywords")
df_exploded["keywords"] = df_exploded["keywords"].str.strip().str.lower()
df_specs = df_exploded[df_exploded["keywords"].isin(valid_specs)].copy()

pivot_df = df_specs.groupby(["keywords", "product"])["sentiment_score"].mean().reset_index()
pivot_table = pivot_df.pivot(index="keywords", columns="product", values="sentiment_score").fillna(0)

pivot_table["Gap_vs_Competitors"] = (
    pivot_table[["Sony XM5", "Momentum 4"]].mean(axis=1) - pivot_table["AirPods Max"]
)
pivot_table["Underperforming"] = pivot_table["Gap_vs_Competitors"] > 0.2

# Display gap table
gap_cols = [
    "AirPods Max", "Sony XM5", "Momentum 4",
    "Gap_vs_Competitors", "Underperforming"
]
print("\n=== GAP ANALYSIS TABLE ===")
display(pivot_table[gap_cols].sort_values("Gap_vs_Competitors", ascending=False))

# ---------------------------------------------
# STEP 4: DESIGN OPPORTUNITY INSIGHTS (TOP GAP ONLY)
# ---------------------------------------------
N = 5  # number of top features to analyze
top_underperf = (
    pivot_table[pivot_table["Underperforming"]]
    .sort_values("Gap_vs_Competitors", ascending=False)
    .head(N)
    .index
    .tolist()
)

comments_by_feature = df_specs[
    (df_specs["product"] == "AirPods Max") &
    (df_specs["keywords"].isin(top_underperf))
].groupby("keywords")["comment"].apply(lambda x: "\n".join(x.dropna().unique()[:10]))

def gpt_design_opportunity(feature, comments):
    prompt = f"""
You are a product strategist. Users gave negative feedback on the **"{feature}"** of the AirPods Max.
Here are sample complaints:
{comments}

Based on these:
1. Identify specific issues
2. Translate to measurable design specs
3. Suggest improvements
4. Estimate potential impact (sales, brand, ROI)
5. List all insights clearly in bullet points.
"""
    try:
        response = openai.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.7,
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {e}"

# Generate insights
#print(f"\n=== GPT DESIGN OPPORTUNITY ANALYSIS (TOP {N} FEATURES) ===")
#for feature, sample_comments in comments_by_feature.items():
 #   print(f"\n\n===== DESIGN OPPORTUNITY: {feature.upper()} =====")
  #  print(gpt_design_opportunity(feature, sample_comments))


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!



=== GAP ANALYSIS TABLE ===


product,AirPods Max,Sony XM5,Momentum 4,Gap_vs_Competitors,Underperforming
keywords,,,,,
microphone,0.333333,0.473684,0.875000,0.341009,True
fit,0.571429,0.851852,0.846154,0.277574,True
build,0.529412,0.800000,0.777778,0.259477,True
latency,0.161290,0.318182,0.500000,0.247801,True
comfort,0.555556,0.742857,0.782609,0.207177,True
case,0.403587,0.471649,0.620253,0.142364,False
battery,0.475410,0.703448,0.523077,0.137853,False
weight,0.378378,0.666667,0.363636,0.136773,False
price,0.481928,0.579545,0.592105,0.103898,False
